In [1]:
#Import necessary libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

In [2]:
# Create lists with machines and machine types:
machine_files = ['./df_6dB_valve_fe_ta_imb.csv', './df_6dB_pump_fe_ta_imb.csv', './df_6dB_slider_fe_ta_imb.csv', './df_6dB_fan_fe_ta_imb.csv']
machine_types = ['id_00', 'id_02', 'id_04', 'id_06']

In [3]:
# Define a function that rund the kmeans algorithm with n_clusters =2 and calculates the correlation between labeled data and the clusters
def get_corr(df, file_path, machine_type):
    df_machine_type = df[df.machine_type == machine_type]
    df_machine_type.reset_index(inplace = True)
    df_machine_type.drop(['machine_type'],axis = 1, inplace = True) 
    df_machine_type_features = df_machine_type.iloc[:,:-1]
    # z-score normalisation per machine
    metrics_normalized_machine_type =(df_machine_type_features-df_machine_type_features.mean())/df_machine_type_features.std()
    metrics_normalized_machine_type = metrics_normalized_machine_type.rename(columns={'mfccs': 'mfccs_zscore','zero_cross': 'zero_cross_zscore','rms': 'rms_zscore'})
    metrics_normalized_machine_type.reset_index(drop=True, inplace = True)
    # fit a kmeans object to the dataset
    kmeans = KMeans(n_clusters=2, init='k-means++').fit(metrics_normalized_machine_type)

    # clusters is an attribute of the object
    cluster_centers = kmeans.cluster_centers_
    # add cluster index to dataframe

    cluster_labels = pd.Series(kmeans.labels_, name='cluster')
    metrics_normalized_machine_type_kmeans = metrics_normalized_machine_type.join(cluster_labels.to_frame())
    metrics_normalized_machine_type_kmeans['target'] = df_machine_type.target
    correlation = abs(metrics_normalized_machine_type_kmeans.target.corr(metrics_normalized_machine_type_kmeans.cluster))
    return {
        'file_path' : file_path,
        'machine_type': machine_type,
        'correlation': correlation
    }
df = pd.read_csv(machine_files[0])
df.drop(['Unnamed: 0'],axis = 1, inplace = True)
get_corr(df, machine_files[0], machine_types[0])

c:\users\kasia\pyproj\machine-monitoring-conditions-unsupervised\my_env\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


{'file_path': './df_6dB_valve_fe_ta_imb.csv',
 'machine_type': 'id_00',
 'correlation': 0.5420905373277507}

In [4]:
# Run the get_corr function for all the machines ad all machine_ids:
corr = []
for file_path in machine_files: 
    #Read and clean data
    df = pd.read_csv(file_path)
    df.drop(['Unnamed: 0'],axis = 1, inplace = True)
    #df = df_original.drop(['target'],axis = 1)
    for machine_type in machine_types:  
        corr.append(get_corr(df, file_path, machine_type))

c:\users\kasia\pyproj\machine-monitoring-conditions-unsupervised\my_env\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
c:\users\kasia\pyproj\machine-monitoring-conditions-unsupervised\my_env\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
c:\users\kasia\pyproj\machine-monitoring-conditions-unsupervised\my_env\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pan

In [5]:
# Create a dataframe with the correlation results
df_corr = pd.DataFrame(corr)

In [7]:
# Create a function that adds a column with mahcne name based on the file_path
def get_machine(row):
    if row.file_path == "./df_6dB_valve_fe_ta_imb.csv":
        return "valve"
    if row.file_path == "./df_6dB_slider_fe_ta_imb.csv":
        return "slider"
    if row.file_path == "./df_6dB_fan_fe_ta_imb.csv":
        return "fan"
    if row.file_path == "./df_6dB_pump_fe_ta_imb.csv":
        return "pump"

In [8]:
df_corr['machine'] = df_corr.apply (lambda row: get_machine(row), axis=1)

In [9]:
df_corr = df_corr[["machine", "machine_type", "correlation","file_path"]]

In [10]:
# Save a dataframe to a .csv file
df_corr.to_csv('kmeans_corr.csv')

In [11]:
df_corr.iloc[:,:-1].sort_values(by = 'correlation', ascending = False, ignore_index=True)

,machine,machine_type,correlation
0,pump,id_04,1.000000
1,slider,id_00,1.000000
2,fan,id_04,0.825811
3,fan,id_06,0.799736
4,pump,id_00,0.796292
5,slider,id_02,0.784465
6,slider,id_04,0.652112
7,pump,id_02,0.587637
8,fan,id_00,0.548124
9,fan,id_02,0.545915


In [12]:
df_corr_mach = df_corr.drop(['machine_type', 'file_path'], axis = 1)

In [13]:
df_corr_mach.groupby('machine').mean()

,correlation
machine,
fan,0.679896
pump,0.729312
slider,0.702915
valve,0.326011
